## 参考链接：http://python.jobbole.com/87181/

In [1]:
import gevent

In [2]:
def test1():
    print(12)
    gevent.sleep(0)
    print(34)
def test2():
    print(56)
    gevent.sleep(0)
    print(78)
# joinall，等待所有的协程执行完毕才结束
gevent.joinall([gevent.spawn(test1), # 创建新的协程对象
                gevent.spawn(test2)])

12
56
34
78


[<Greenlet "Greenlet-0" at 0x106dc9948: _run>,
 <Greenlet "Greenlet-1" at 0x106dc9a48: _run>]

In [4]:
import socket
urls = ['www.baidu.com', 'www.gevent.org', 'www.python.org']
jobs = [gevent.spawn(socket.gethostbyname, url) for url in urls]
gevent.joinall(jobs, timeout=5)

print([job.value for job in jobs])

['111.13.100.92', '23.100.69.251', '151.101.24.223']


In [5]:
from gevent import monkey
monkey.patch_socket() # 将标准库替换成gevent非阻塞模式，也可以使用monkey.patch_all()
urls = ['www.baidu.com', 'www.gevent.org', 'www.python.org']
jobs = [gevent.spawn(socket.gethostbyname, url) for url in urls]
gevent.joinall(jobs, timeout=5)

print([job.value for job in jobs])

['111.13.100.91', '23.100.69.251', '151.101.24.223']


In [9]:
# 获取协程的状态
def win():
    return 'you win!'
def fail():
    raise Exception('you failed!')
winner = gevent.spawn(win)
loser = gevent.spawn(fail)

print(winner.started)
print(loser.started)

print(winner.ready())
print(loser.ready())

try:
    # greenlet中的异常，不会抛出到外面
    gevent.joinall([winner, loser])
except Exception as e:
    print('never be reaced')

print(winner.ready())
print(loser.ready())

print(winner.value)
print(loser.value)

print(winner.successful())
print(loser.successful())

print(loser.exception)

True
True
False
False
True
True
you win!
None
True
False
you failed!


Traceback (most recent call last):
  File "src/gevent/greenlet.py", line 705, in gevent._greenlet.Greenlet.run
  File "<ipython-input-9-83cacf7d944e>", line 5, in fail
    raise Exception('you failed!')
Exception: you failed!
2018-10-25T02:17:31Z <Greenlet "Greenlet-2" at 0x107120548: fail> failed with Exception



In [10]:
# 协程超时
from gevent import Timeout

timeout = Timeout(2)
timeout.start()

def wait():
    print('gevent')
    gevent.sleep(10)

print('main')
try:
    gevent.spawn(wait).join()
except Timeout:
    print('could not complete')

main
gevent
could not complete


In [11]:
# 在with块中超时有效
with Timeout(1):
    gevent.sleep(10)
    try:
        gevent.spawn(wait).join()
    except Timeout:
        print('could not complete')

try:
    gevent.spawn(wait).join()
except Timeout:
    print('could not complete')

could not complete
gevent
gevent


In [12]:
# 协程的通讯，wait()阻塞当前协程，set召唤之前阻塞的协程
from gevent.event import Event

evt = Event()
def setter():
    print('wait')
    gevent.sleep(3)
    print('ok')
    evt.set()
def waiter():
    print('waiting')
    evt.wait()
    print('finish')
gevent.joinall([gevent.spawn(setter),
               gevent.spawn(waiter),
               gevent.spawn(waiter),
               gevent.spawn(waiter),
               gevent.spawn(waiter),
               gevent.spawn(waiter)])

wait
waiting
waiting
waiting
waiting
waiting
ok
finish
finish
finish
finish
finish


[<Greenlet "Greenlet-3" at 0x1072ffc48: _run>,
 <Greenlet "Greenlet-4" at 0x107632248: _run>,
 <Greenlet "Greenlet-5" at 0x107632548: _run>,
 <Greenlet "Greenlet-6" at 0x107632348: _run>,
 <Greenlet "Greenlet-7" at 0x1072ffd48: _run>,
 <Greenlet "Greenlet-8" at 0x107632448: _run>]

In [13]:
# 异步事件
from gevent.event import AsyncResult
aevt = AsyncResult()

def setter():
    print('wait')
    gevent.sleep(3)
    print('ok')
    aevt.set('hello!')
def waiter():
    print('wait')
    message = aevt.get()
    print('get %s' % message)

gevent.joinall([gevent.spawn(setter),
               gevent.spawn(waiter),
               gevent.spawn(waiter),
               gevent.spawn(waiter),
               gevent.spawn(waiter),
               gevent.spawn(waiter)])    

wait
wait
wait
wait
wait
wait
ok
get hello!
get hello!
get hello!
get hello!
get hello!


[<Greenlet "Greenlet-9" at 0x107632148: _run>,
 <Greenlet "Greenlet-10" at 0x107632a48: _run>,
 <Greenlet "Greenlet-11" at 0x107632848: _run>,
 <Greenlet "Greenlet-12" at 0x107632748: _run>,
 <Greenlet "Greenlet-13" at 0x107632648: _run>,
 <Greenlet "Greenlet-14" at 0x107632948: _run>]

In [14]:
# 队列
from gevent.queue import Queue

products = Queue()

def consumer(name):
    while not products.empty():
        print('%s got product %s' % (name, products.get()))
        gevent.sleep(0)
    print('%s quit' % name)
def producer():
    for i in range(1, 10):
        products.put(i)
gevent.joinall([
    gevent.spawn(producer),
    gevent.spawn(consumer, '1'),
    gevent.spawn(consumer, '2'),
    gevent.spawn(consumer, '3')
])

1 got product 1
2 got product 2
3 got product 3
1 got product 4
2 got product 5
3 got product 6
1 got product 7
2 got product 8
3 got product 9
1 quit
2 quit
3 quit


[<Greenlet "Greenlet-15" at 0x107632b48: _run>,
 <Greenlet "Greenlet-16" at 0x107632c48: _run>,
 <Greenlet "Greenlet-17" at 0x107632d48: _run>,
 <Greenlet "Greenlet-18" at 0x107632e48: _run>]

In [15]:
# 信号量
from gevent.coros import BoundedSemaphore
sem = BoundedSemaphore(2)

def worker(n):
    sem.acquire()
    print('worker %i acquire semaphore' % n)
    gevent.sleep(0)
    sem.release()
    print('worker %i released semaphore' % n)
gevent.joinall([
    gevent.spawn(worker, i) for i in range(0, 6)
])

ModuleNotFoundError: No module named 'gevent.coros'

In [16]:
# 协程本地变量, 作用域限制在协程内
from gevent.local import local

data = local()

def f1():
    data.x = 1
    print(data.x)
def f2():
    try:
        print(data.x)
    except AttributeError:
        print('x is not visible')
gevent.joinall([
    gevent.spawn(f1), gevent.spawn(f2)
])

1
x is not visible


[<Greenlet "Greenlet-19" at 0x1076d4048: _run>,
 <Greenlet "Greenlet-20" at 0x1076d4148: _run>]